# 🔗 LangChain Avanzado - Agentes y Herramientas
## Clase 2 - Lazarus

En esta clase profundizaremos en:
1. **Chains**: Cadenas personalizadas y composición
2. **Memory**: Gestión de memoria conversacional
3. **Agents**: Agentes autónomos con herramientas
4. **Tools**: Creación de herramientas personalizadas
5. **Output Parsers**: Estructuración de respuestas

## 📦 Instalación de dependencias

Ejecuta esta celda una sola vez

In [ ]:
!pip install openai python-dotenv langchain langchain-openai langchain-community \
    langchain-core langchain-classic pypdf faiss-cpu wikipedia duckduckgo-search requests -q

## ⚙️ Configuración inicial

In [ ]:
import os
from dotenv import load_dotenv

# Cargamos variables de entorno
load_dotenv()

# Verificamos la API key
api_key = os.getenv('OPENAI_API_KEY')
if not api_key:
    raise ValueError("⚠️ No se encontró OPENAI_API_KEY en el archivo .env")

print("✅ Configuración cargada correctamente")

---
# 🔗 PARTE 1: Chains (Cadenas)

Las **Chains** son secuencias de componentes que procesan datos de forma encadenada.

### Tipos de Chains:
- **LLMChain**: Básica (prompt + LLM)
- **SequentialChain**: Encadena múltiples chains
- **RouterChain**: Enruta a diferentes chains según la entrada
- **TransformChain**: Transforma datos antes de pasar al LLM

## 1.1 LLMChain Básica

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate

# Configuramos el LLM
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.7)

# Creamos un prompt template
prompt = PromptTemplate(
    input_variables=["producto", "caracteristicas"],
    template="""Eres un copywriter experto. Crea una descripción de producto atractiva.

Producto: {producto}
Características: {caracteristicas}

Descripción persuasiva:"""
)

# Creamos la chain usando LCEL (LangChain Expression Language)
chain_descripcion = prompt | llm

# Ejecutamos
resultado = chain_descripcion.invoke({
    "producto": "Rotomartillo HILTI TE-70",
    "caracteristicas": "Potente, ideal para concreto, marca profesional"
})

print("🎯 DESCRIPCIÓN GENERADA:")
print(resultado.content)

## 1.2 SequentialChain - Cadenas en Secuencia

Encadenamos múltiples operaciones:

In [ ]:
from langchain_core.output_parsers import StrOutputParser

# Chain 1: Generar características técnicas
prompt1 = PromptTemplate(
    input_variables=["producto"],
    template="Lista 5 características técnicas clave de: {producto}"
)
chain1 = prompt1 | llm | StrOutputParser()

# Chain 2: Crear descripción de venta
prompt2 = PromptTemplate(
    input_variables=["caracteristicas"],
    template="""Convierte estas características técnicas en un pitch de ventas atractivo:
{caracteristicas}

Pitch (máximo 50 palabras):"""
)
chain2 = prompt2 | llm | StrOutputParser()

# Combinamos las chains usando LCEL
from operator import itemgetter

cadena_completa = (
    {"caracteristicas": chain1}
    | chain2
)

# Ejecutamos
print("\n" + "="*60)
print("🔄 Ejecutando cadena secuencial...")
resultado_final = cadena_completa.invoke({"producto": "Compactador de Rodillo 524 KGS"})
print("\n🎯 RESULTADO FINAL:")
print(resultado_final)

---
# 💭 PARTE 2: Memory (Memoria Conversacional)

La **memoria** permite que el agente recuerde conversaciones anteriores.

### Tipos de Memoria:
- **ConversationBufferMemory**: Almacena todo el historial
- **ConversationBufferWindowMemory**: Solo las últimas K conversaciones
- **ConversationSummaryMemory**: Resume conversaciones antiguas
- **ConversationEntityMemory**: Recuerda entidades mencionadas

## 2.1 Memory Básica - Buffer Memory

In [ ]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_community.chat_message_histories import ChatMessageHistory

# Creamos memoria
store = {}

def get_session_history(session_id: str):
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

# Creamos el prompt con memoria
prompt_memoria = ChatPromptTemplate.from_messages([
    ("system", "Eres un asistente amable y profesional."),
    MessagesPlaceholder(variable_name="history"),
    ("human", "{input}")
])

# Creamos la chain con memoria
chain_con_memoria = prompt_memoria | llm

conversacion = RunnableWithMessageHistory(
    chain_con_memoria,
    get_session_history,
    input_messages_key="input",
    history_messages_key="history"
)

# Simulamos una conversación
print("\n" + "="*60)
print("🗣️ CONVERSACIÓN CON MEMORIA")
print("="*60)

session_id = "sesion_1"

resp1 = conversacion.invoke(
    {"input": "Hola, me llamo Juan y necesito rentar un demoledor"},
    config={"configurable": {"session_id": session_id}}
)
print(f"\n🤖 Respuesta 1: {resp1.content}")

resp2 = conversacion.invoke(
    {"input": "¿Cuál me recomiendas para concreto pesado?"},
    config={"configurable": {"session_id": session_id}}
)
print(f"\n🤖 Respuesta 2: {resp2.content}")

resp3 = conversacion.invoke(
    {"input": "¿Recuerdas mi nombre?"},
    config={"configurable": {"session_id": session_id}}
)
print(f"\n🤖 Respuesta 3: {resp3.content}")

# Inspeccionamos la memoria
print("\n📝 HISTORIAL DE CONVERSACIÓN:")
history = get_session_history(session_id)
for msg in history.messages:
    print(f"  {msg.type}: {msg.content[:100]}...")

## 2.2 Window Memory - Solo las últimas K conversaciones

In [ ]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory

# Memoria con ventana (solo últimas K mensajes)
store_ventana = {}

def get_window_history(session_id: str):
    if session_id not in store_ventana:
        store_ventana[session_id] = ChatMessageHistory()
    return store_ventana[session_id]

# Helper para mantener solo últimos K mensajes
def trim_messages(messages, k=4):  # k=4 significa 2 interacciones (2 humano + 2 AI)
    return messages[-k:]

prompt_ventana = ChatPromptTemplate.from_messages([
    ("system", "Eres un asistente amable."),
    MessagesPlaceholder(variable_name="history"),
    ("human", "{input}")
])

chain_ventana = prompt_ventana | llm

conversacion_ventana = RunnableWithMessageHistory(
    chain_ventana,
    get_window_history,
    input_messages_key="input",
    history_messages_key="history"
)

print("\n" + "="*60)
print("🪟 CONVERSACIÓN CON VENTANA DE MEMORIA (k=2)")
print("="*60)

session_id_2 = "sesion_ventana"

conversacion_ventana.invoke(
    {"input": "Me llamo María"},
    config={"configurable": {"session_id": session_id_2}}
)

conversacion_ventana.invoke(
    {"input": "Vivo en Honduras"},
    config={"configurable": {"session_id": session_id_2}}
)

conversacion_ventana.invoke(
    {"input": "Necesito un compresor"},
    config={"configurable": {"session_id": session_id_2}}
)

# Trimear mensajes manualmente
history = get_window_history(session_id_2)
history.messages = trim_messages(history.messages, k=4)

# Esta pregunta debe fallar porque "María" ya salió de la ventana
resp = conversacion_ventana.invoke(
    {"input": "¿Recuerdas mi nombre?"},
    config={"configurable": {"session_id": session_id_2}}
)
print(f"\n🤖 {resp.content}")

print("\n💡 Solo recuerda las últimas 2 interacciones, por eso no recuerda el nombre")

## 2.3 Summary Memory - Resume conversaciones largas

In [ ]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_community.chat_message_histories import ChatMessageHistory

# Para Summary Memory, necesitamos simular el resumen manualmente
# ya que ConversationSummaryMemory está deprecado

print("\n" + "="*60)
print("📄 CONVERSACIÓN CON RESUMEN AUTOMÁTICO")
print("="*60)

# Simulamos conversaciones
conversaciones = [
    {"humano": "Soy Pedro, gerente de construcción de CONCESA", "ai": "Mucho gusto Pedro, ¿en qué puedo ayudarte hoy?"},
    {"humano": "Estamos construyendo un edificio de 5 pisos", "ai": "Excelente proyecto. ¿Qué tipo de equipos necesitas?"},
    {"humano": "Necesitamos equipos para 3 meses", "ai": "Perfecto, tenemos opciones de renta a largo plazo con descuentos."},
    {"humano": "El presupuesto es limitado", "ai": "Entiendo, podemos buscar las opciones más económicas para ti."}
]

# Creamos un prompt para resumir
from langchain_core.output_parsers import StrOutputParser

prompt_resumen = ChatPromptTemplate.from_messages([
    ("system", "Resume la siguiente conversación de forma concisa:"),
    ("human", "{conversacion}")
])

chain_resumen = prompt_resumen | llm | StrOutputParser()

# Convertir conversaciones a texto
texto_conversacion = "\n".join([
    f"Cliente: {c['humano']}\nAsistente: {c['ai']}" 
    for c in conversaciones
])

# Generar resumen
resumen = chain_resumen.invoke({"conversacion": texto_conversacion})

print("\n📊 RESUMEN DE LA CONVERSACIÓN:")
print(resumen)

print("\n💡 El LLM resumió toda la conversación en vez de almacenar todo")
print("\n📝 Conversación original:")
for i, c in enumerate(conversaciones, 1):
    print(f"{i}. Cliente: {c['humano']}")
    print(f"   Asistente: {c['ai']}")

---
# 🤖 PARTE 3: Agents (Agentes Autónomos)

Los **Agents** son sistemas que pueden:
- Decidir qué herramientas usar
- Razonar sobre problemas
- Ejecutar acciones de forma autónoma

### Tipos de Agentes:
- **Zero-shot React**: Decide herramientas sin ejemplos
- **Conversational**: Con memoria conversacional
- **OpenAI Functions**: Usa function calling de OpenAI

## 3.1 Crear Herramientas Personalizadas

In [ ]:
from langchain_core.tools import tool
from datetime import datetime

# Usamos el decorador @tool para crear herramientas (forma moderna)

@tool
def calcular_descuento(precio_dias: str) -> str:
    """Calcula descuento por volumen. Input: 'precio,dias' ejemplo: '500,10'"""
    try:
        precio, dias = precio_dias.split(',')
        precio = float(precio)
        dias = int(dias)
        
        total = precio * dias
        
        # Descuentos por volumen
        if dias >= 30:
            descuento = 0.20
        elif dias >= 14:
            descuento = 0.15
        elif dias >= 7:
            descuento = 0.10
        else:
            descuento = 0
        
        total_con_descuento = total * (1 - descuento)
        ahorro = total - total_con_descuento
        
        return f"Total sin descuento: L{total:.2f}\nDescuento aplicado: {descuento*100}%\nTotal con descuento: L{total_con_descuento:.2f}\nAhorro: L{ahorro:.2f}"
    except:
        return "Error: Formato debe ser 'precio,dias' ejemplo: '500,10'"

@tool
def verificar_disponibilidad(equipo: str) -> str:
    """Verifica si un equipo está disponible. Input: nombre del equipo"""
    # Base de datos simulada
    inventario = {
        "demoledor": {"disponible": True, "unidades": 3},
        "rotomartillo": {"disponible": True, "unidades": 5},
        "compactador": {"disponible": False, "unidades": 0},
        "mezcladora": {"disponible": True, "unidades": 2}
    }
    
    equipo_lower = equipo.lower()
    for key in inventario:
        if key in equipo_lower:
            info = inventario[key]
            if info["disponible"]:
                return f"✅ {equipo} está DISPONIBLE. Unidades en stock: {info['unidades']}"
            else:
                return f"❌ {equipo} NO está disponible actualmente. Stock: {info['unidades']}"
    
    return f"⚠️ No encontré información sobre '{equipo}' en el inventario"

@tool
def calcular_fecha_entrega(dias: str) -> str:
    """Calcula la fecha de entrega. Input: número de días de renta"""
    try:
        from datetime import timedelta
        dias = int(dias)
        fecha_entrega = datetime.now() + timedelta(days=dias)
        return f"Si rentas por {dias} días, la fecha de devolución sería: {fecha_entrega.strftime('%d/%m/%Y')}"
    except:
        return "Error: Ingresa un número válido de días"

print("✅ Herramientas personalizadas creadas con decorador @tool")

## 3.2 Crear el Agente con Herramientas

In [ ]:
from langchain_classic.agents import create_react_agent, AgentExecutor
from langchain_core.prompts import PromptTemplate

# Las herramientas ya están creadas con el decorador @tool en la celda anterior
tools = [
    calcular_descuento,
    verificar_disponibilidad,
    calcular_fecha_entrega
]

# Template para el agente ReAct
template = '''Responde a las siguientes preguntas lo mejor que puedas. Tienes acceso a las siguientes herramientas:

{tools}

Usa el siguiente formato:

Question: la pregunta de entrada que debes responder
Thought: siempre debes pensar qué hacer
Action: la acción a tomar, debe ser una de [{tool_names}]
Action Input: el input para la acción
Observation: el resultado de la acción
... (este Thought/Action/Action Input/Observation puede repetirse N veces)
Thought: Ahora sé la respuesta final
Final Answer: la respuesta final a la pregunta de entrada original

Comienza!

Question: {input}
Thought: {agent_scratchpad}'''

prompt_agente = PromptTemplate.from_template(template)

# Crear el agente
agente = create_react_agent(llm, tools, prompt_agente)

# Crear el executor
agente_executor = AgentExecutor(
    agent=agente,
    tools=tools,
    verbose=True,
    max_iterations=5,
    handle_parsing_errors=True
)

print("\n✅ Agente creado con 3 herramientas usando ReAct")

## 3.3 Demo del Agente en Acción

In [ ]:
print("\n" + "="*60)
print("🤖 AGENTE AUTÓNOMO EN ACCIÓN")
print("="*60)

# Pregunta 1: Verificar disponibilidad
print("\n👤 Usuario: ¿Tienen rotomartillos disponibles?")
resp1 = agente_executor.invoke({"input": "¿Tienen rotomartillos disponibles?"})
print(f"🤖 Agente: {resp1['output']}")

# Pregunta 2: Calcular precio con descuento
print("\n👤 Usuario: ¿Cuánto me costaría rentar un demoledor por 15 días a L550 por día?")
resp2 = agente_executor.invoke({"input": "¿Cuánto me costaría rentar un demoledor por 15 días a L550 por día?"})
print(f"🤖 Agente: {resp2['output']}")

# Pregunta 3: Fecha de entrega
print("\n👤 Usuario: Si lo rento por 15 días, ¿cuándo lo tengo que devolver?")
resp3 = agente_executor.invoke({"input": "Si lo rento por 15 días, ¿cuándo lo tengo que devolver?"})
print(f"🤖 Agente: {resp3['output']}")

print("\n💡 El agente decidió automáticamente qué herramientas usar para cada pregunta")

---
# 🛠️ PARTE 4: Output Parsers (Estructurar Respuestas)

Los **Output Parsers** permiten que el LLM devuelva datos estructurados (JSON, listas, objetos).

## 4.1 Structured Output Parser

In [ ]:
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate
from pydantic import BaseModel, Field

# Definimos el esquema con Pydantic
class ProductoSchema(BaseModel):
    nombre_producto: str = Field(description="Nombre del producto")
    precio: str = Field(description="Precio por día en Lempiras")
    categoria: str = Field(description="Categoría del producto")
    recomendado_para: str = Field(description="Para qué tipo de trabajo es recomendado")

# Creamos el parser
output_parser = JsonOutputParser(pydantic_object=ProductoSchema)

# Creamos el prompt
prompt = PromptTemplate(
    template="""Extrae la información del siguiente producto y devuélvela en formato JSON.

{format_instructions}

Producto: {producto}
""",
    input_variables=["producto"],
    partial_variables={"format_instructions": output_parser.get_format_instructions()}
)

# Creamos la chain
chain_estructurada = prompt | llm | output_parser

# Ejecutamos
resultado = chain_estructurada.invoke({
    "producto": "Demoledor TE-3000, martillo rompedor excepcional para demolición pesada de concreto, L1,100.00 por día"
})

print("\n📊 SALIDA ESTRUCTURADA (JSON):")
print(resultado)
print("\n💡 Ahora podemos usar estos datos programáticamente")
print(f"   Nombre: {resultado['nombre_producto']}")
print(f"   Precio: {resultado['precio']}")

## 4.2 Pydantic Output Parser (Validación con Tipos)

In [ ]:
from langchain_core.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field
from typing import List

# Definimos el modelo con Pydantic
class Producto(BaseModel):
    nombre: str = Field(description="Nombre del producto")
    precio: float = Field(description="Precio por día en Lempiras")
    caracteristicas: List[str] = Field(description="Lista de características principales")
    disponible: bool = Field(description="Si está disponible o no")

# Creamos el parser
parser = PydanticOutputParser(pydantic_object=Producto)

# Creamos el prompt
prompt = PromptTemplate(
    template="""Analiza este producto y devuelve la información estructurada.

{format_instructions}

Descripción: {descripcion}
""",
    input_variables=["descripcion"],
    partial_variables={"format_instructions": parser.get_format_instructions()}
)

# Chain
chain_pydantic = prompt | llm | parser

# Ejecutamos
producto = chain_pydantic.invoke({
    "descripcion": """Rotomartillo TE-70 HILTI, martillo perforador muy potente para 
    tareas de carga pesada de taladro y cincelado en concreto. Precio L750 por día. 
    Actualmente disponible en stock."""
})

print("\n🎯 OBJETO PYDANTIC VALIDADO:")
print(f"Nombre: {producto.nombre}")
print(f"Precio: L{producto.precio}")
print(f"Características: {producto.caracteristicas}")
print(f"Disponible: {'Sí' if producto.disponible else 'No'}")

print("\n💡 Pydantic valida automáticamente los tipos de datos")

---
# 🎯 PARTE 5: Proyecto Final - Agente Completo de Ventas

Combinamos todo lo aprendido en un agente de ventas inteligente con:
- ✅ Memoria conversacional
- ✅ Herramientas personalizadas
- ✅ RAG para consultar catálogo
- ✅ Output estructurado

In [ ]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_core.tools import tool
from langchain_classic.agents import create_react_agent, AgentExecutor

print("🏗️ CONSTRUYENDO AGENTE DE VENTAS COMPLETO...\n")

# 1. Cargamos el catálogo PDF (del notebook anterior)
PDF_PATH = 'Documentos - PDF/Catalogo_Equipos_Construccion.pdf'

if os.path.exists(PDF_PATH):
    print("📚 Cargando catálogo PDF...")
    loader = PyPDFLoader(PDF_PATH)
    documents = loader.load()
    
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=500,
        chunk_overlap=100
    )
    docs = text_splitter.split_documents(documents)
    
    embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
    vectorstore = FAISS.from_documents(docs, embeddings)
    
    print(f"✅ Catálogo cargado: {len(docs)} chunks")
    
    # Creamos herramienta de búsqueda en catálogo usando decorador @tool
    @tool
    def buscar_catalogo(query: str) -> str:
        """Busca productos en el catálogo. Input: descripción del producto"""
        docs = vectorstore.similarity_search(query, k=2)
        resultados = "\n\n".join([doc.page_content for doc in docs])
        return f"Productos encontrados:\n{resultados}"
    
    # Actualizamos las herramientas
    tools_completo = tools + [buscar_catalogo]
    
    # Template mejorado para agente de ventas
    template_ventas = '''Eres un agente de ventas experto de CONCESA que ayuda a clientes a encontrar equipos de construcción.
Siempre eres amable, profesional y buscas cerrar ventas.

Tienes acceso a las siguientes herramientas:

{tools}

Usa el siguiente formato:

Question: la pregunta del cliente
Thought: siempre piensa qué necesita el cliente
Action: la acción a tomar, debe ser una de [{tool_names}]
Action Input: el input para la acción
Observation: el resultado de la acción
... (puedes repetir Thought/Action/Action Input/Observation varias veces)
Thought: Ahora tengo la información completa para responder
Final Answer: tu respuesta profesional y útil al cliente

Comienza!

Question: {input}
Thought: {agent_scratchpad}'''

    prompt_ventas = PromptTemplate.from_template(template_ventas)
    
    # Crear agente de ventas
    agente_ventas_react = create_react_agent(llm, tools_completo, prompt_ventas)
    
    agente_ventas = AgentExecutor(
        agent=agente_ventas_react,
        tools=tools_completo,
        verbose=True,
        max_iterations=5,
        handle_parsing_errors=True
    )
    
    print("✅ Agente de ventas completo creado con 4 herramientas")
else:
    print("⚠️ No se encontró el PDF. El agente funcionará sin catálogo.")
    agente_ventas = agente_executor  # Usamos el agente anterior

## 5.1 Demo Final - Agente de Ventas Completo

In [ ]:
print("\n" + "="*70)
print("🎬 DEMO FINAL - AGENTE DE VENTAS INTELIGENTE")
print("="*70)

# Simulamos una conversación de ventas completa
conversacion_demo = [
    "Hola, necesito equipos para demoler concreto",
    "¿Cuál es el más potente que tienen?",
    "Perfecto, lo necesito por 20 días. ¿Cuánto sería el total con el TE-3000?",
    "¿Está disponible para entrega inmediata?"
]

for i, pregunta in enumerate(conversacion_demo, 1):
    print(f"\n👤 Cliente: {pregunta}")
    respuesta = agente_ventas.invoke({"input": pregunta})
    print(f"\n🤖 Agente: {respuesta['output']}")
    print("\n" + "-"*70)

print("\n✨ El agente usó automáticamente:")
print("   1. Búsqueda en catálogo (RAG)")
print("   2. Cálculo de descuentos")
print("   3. Verificación de disponibilidad")
print("   4. Razonamiento paso a paso (ReAct)")

---
# 📚 Resumen de Conceptos Clave

## Lo que aprendimos hoy:

### 1. **Chains** (Cadenas)
- Encadenar operaciones de forma secuencial
- Componer workflows complejos
- `LLMChain`, `SequentialChain`

### 2. **Memory** (Memoria)
- Mantener contexto conversacional
- Diferentes estrategias: Buffer, Window, Summary
- Fundamental para chatbots

### 3. **Agents** (Agentes)
- Sistemas autónomos que deciden qué hacer
- Usan herramientas según la necesidad
- Razonamiento ReAct (Reason + Act)

### 4. **Tools** (Herramientas)
- Extender capacidades del LLM
- Herramientas personalizadas
- Integración con APIs, bases de datos, etc.

### 5. **Output Parsers**
- Estructurar respuestas del LLM
- JSON, Pydantic, listas
- Validación de tipos

### 6. **Integración RAG + Agents**
- Combinar búsqueda vectorial con agentes
- Sistema completo de atención al cliente
- Escalable y mantenible

---
# 🚀 Ejercicio Final

**Reto**: Crea tu propio agente personalizado que:
1. Tenga al menos 2 herramientas personalizadas
2. Use memoria conversacional
3. Devuelva salidas estructuradas

Ideas:
- Agente de soporte técnico
- Asistente de investigación
- Generador de reportes
- Planificador de proyectos

In [ ]:
# 👨‍💻 TU CÓDIGO AQUÍ
# Crea tu agente personalizado



---
# 📖 Recursos Adicionales

- [Documentación LangChain](https://python.langchain.com/docs/get_started/introduction)
- [LangChain Agents](https://python.langchain.com/docs/modules/agents/)
- [LangChain Memory](https://python.langchain.com/docs/modules/memory/)
- [LangChain Tools](https://python.langchain.com/docs/modules/tools/)
- [Output Parsers](https://python.langchain.com/docs/modules/model_io/output_parsers/)

---
## ✅ Fin de la Clase 2

**¿Preguntas?**